# Climatologie de l'atmosphère en moyenne zonale

Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2021

L’objectif de ces activités Python est de mettre en évidence les grands traits de la circulation générale atmosphérique **en moyenne zonale** à travers l’étude de climatologies saisonnières des paramètres atmosphériques température, vent, humidité et précipitations.

Les fichiers de données au format netcdf (moyennes mensuelles NCEP/NCAR) doivent être téléchargés et placés dans le répertoire data :

Température, vent zonal, vent méridien, vitesse verticale, humidité spécifique :
- ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/air.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/uwnd.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/vwnd.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/omega.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/shum.mon.mean.nc


Précipitations :
- ftp://ftp.cdc.noaa.gov/Datasets/gpcp/precip.mon.mean.nc

Concepts Python illustrés :
- Exploitation de donnnées météorologiques au format netcdf (xarray)
- Calcul de climatologies saisonnières (xarray)
- Tracé de coupes et profils (matplotlib)

In [ ]:
%matplotlib inline

import os

import xarray as xr
import numpy as np
import netCDF4

import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.path as mpath

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

import warnings
warnings.filterwarnings('ignore')

# Traitement des données

In [ ]:
year1='1990'
year2='2019'

In [ ]:
diri="./data/"
ft    = xr.open_dataset(diri+"air.mon.mean.nc").sel(time=slice(year1,year2))+273.15
fu    = xr.open_dataset(diri+"uwnd.mon.mean.nc").sel(time=slice(year1,year2))
fv    = xr.open_dataset(diri+"vwnd.mon.mean.nc").sel(time=slice(year1,year2))
fw    = xr.open_dataset(diri+"omega.mon.mean.nc").sel(time=slice(year1,year2))
fq    = xr.open_dataset(diri+"shum.mon.mean.nc").sel(time=slice(year1,year2))/1000
fpr = xr.open_dataset(diri+"precip.mon.mean.nc").sel(time=slice(year1,year2))

# Latitudes, niveaux verticaux
lat  = ft.lat.values
lats=fpr.lat.values
lev = ft.level.values
levw = fw.level.values
levq = fq.level.values

print(ft)
print(fw)

In [ ]:
seasons=['DJF','JJA','MAM','SON']
months=['Jan','Feb','Mar','Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

ft_mean = ft.groupby('time.season').mean('time')
fu_mean = fu.groupby('time.season').mean('time')
fv_mean = fv.groupby('time.season').mean('time')
fw_mean = fw.groupby('time.season').mean('time')
fq_mean = fq.groupby('time.season').mean('time')
fpr_mean = fpr.groupby('time.season').mean('time')

t0 = ft_mean['air']
u0 = fu_mean['uwnd']
v0 = fv_mean['vwnd']
w = fw_mean['omega']
q0 = fq_mean['shum']*1000
pr0 = fpr_mean['precip']

print(t0.shape) # saison,niveau,lat,lon

In [ ]:
cp=1004.5
r=2*cp/7
kap=r/cp
theta=t0*(1.E5/(lev[np.newaxis,:,np.newaxis,np.newaxis]*100))**kap

In [ ]:
# Moyennes zonales
tz = t0.mean('lon')
uz = u0.mean('lon')
vz = v0.mean('lon')
wz = w.mean('lon')
qz = q0.mean('lon')
prz=pr0.mean('lon')
thetaz = theta.mean('lon')
print(tz.shape) # saison,niveau,lat

# Moyennes annuelles
uz_annual=uz.mean('season')
vz_annual=vz.mean('season')
wz_annual=wz.mean('season')
prz_annual=prz.mean('season')

print(uz_annual.shape) # level,lat

# Fonctions graphiques

In [ ]:
# on s'arrête à 100hPa sur la varticale
def plot_zonal_mean(ax):
    ax.set_yscale('symlog')
    ax.set_ylim(1000, 100)
    ax.set_xticks(np.arange(-90, 120, 30))
    ax.set_yticks(np.arange(1000, 0, -100))
    ax.tick_params(labelsize=8)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.set_xlabel('Latitude')
    ax.set_ylabel('Pressure level')
    return ax

# on s'arrête à 10hPa sur la verticale
def plot_zonal_mean2(ax):
    ax.set_yscale('symlog')
    ax.set_ylim(1000, 10)
    ax.set_xticks(np.arange(-90, 120, 30))
    ax.set_yticks(np.arange(1000, 0, -100))
    ax.tick_params(labelsize=8)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.set_xlabel('Latitude')
    ax.set_ylabel('Pressure level')
    return ax

In [ ]:
levels_tz = np.arange(180,320,10)
levels_thetaz1 = np.arange(200,920,20)
levels_thetaz2 = np.arange(200,510,10)
levels_uz1 = np.arange(-40,42.5,2.5)
levels_uz2 = np.arange(-50,52.5,2.5)
levels_uz_an = np.arange(0,55,5)
levels_wz = np.arange(-0.05,0.052,0.002)
levels_vz =[-7.0, -6.5, -6.0, -5.5, -5.0, -4.5, -4.0, -3.5, -3.0,
 -2.5, -2.0, -1.5, -1.0, -0.5, 0.5, 1.0, 1.5, 2.0, 2.5,
  3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7]
levels_qz = np.arange(0,20,1)

# Température

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=1, figsize=(15, 15), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Temperature - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    plot_zonal_mean2(ax)
    cf = ax.contourf(lat, lev, tz[i,:,:], levels_tz, cmap='jet', extend='both')
    c = ax.contour(lat, lev, tz[i,:,:], levels_tz, colors='black', linewidths=1)
    plt.clabel(c, levels_tz, fmt='%1.2i')
 
cb = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cb.set_label('mgp', size='small')

figname='./figs/t_zmean_climatology'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Vent zonal

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=1, figsize=(15, 15), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Zonal wind - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    plot_zonal_mean(ax)
    cf = ax.contourf(lat, lev, uz[i,:,:], levels_uz1, cmap='jet', extend='both')
    c = ax.contour(lat, lev, uz[i,:,:], levels_uz1, colors='black', linewidths=1)
    plt.clabel(c, levels_uz1, fmt='%2.1i')
 
cb = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cb.set_label('m/s', size='small')

figname='./figs/u_zmean_climatology'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=1, figsize=(15, 15), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Zonal wind - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    plot_zonal_mean2(ax)
    cf = ax.contourf(lat, lev, uz[i,:,:], levels_uz2, cmap='jet', extend='both')
    c = ax.contour(lat, lev, uz[i,:,:], levels_uz2, colors='black', linewidths=1)
    plt.clabel(c, levels_uz2, fmt='%2.1i')

cb = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cb.set_label('m/s', size='small')

figname='./figs/u_zmean_climatology2'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Température potentielle

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=1, figsize=(15, 15), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Potential temperature - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    plot_zonal_mean2(ax)
    cf = ax.contourf(lat, lev, thetaz[i,:,:], levels_thetaz1, cmap='jet', extend='both')
    c2 = ax.contour(lat, lev, thetaz[i,:,:], levels_thetaz1, colors='black', linewidths=1)
    plt.clabel(c2, levels_thetaz1, fmt='%1.2i')

cf = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cf.set_label('K', size='small')

fig.set_constrained_layout_pads(w_pad=0., h_pad=0.1, hspace=0., wspace=0.)

figname='./figs/theta_zmean_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Equilibre du vent thermique de la circulation en moyenne zonale

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=1, figsize=(15, 15), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Zonal wind and potential temperature - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    plot_zonal_mean(ax)
    cf = ax.contourf(lat, lev, uz[i,:,:], levels_uz1, cmap='jet', extend='both')
    c2 = ax.contour(lat, lev, thetaz[i,:,:], levels_thetaz2, colors='black', linewidths=1)
    plt.clabel(c2, levels_thetaz2, fmt='%1.2i')

cf = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cf.set_label('K', size='small')

fig.set_constrained_layout_pads(w_pad=0., h_pad=0.1, hspace=0., wspace=0.)

figname='./figs/theta_u_zmean_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
tz_mean=tz.mean(axis=1)
thetaz_mean=thetaz.mean(axis=1)

fig = plt.figure(figsize=(15., 8.))
fig.suptitle('Temperature and potential temperature - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)
ax = fig.add_subplot(1, 1, 1)
plot_zonal_mean(ax)
c1 = ax.contour(lat, lev, tz[i,:,:], levels_tz, colors='blue', linewidths=1)
plt.clabel(c1, levels_tz, fmt='%1.2i')
c2 = ax.contour(lat, lev, thetaz[i,:,:], levels_thetaz2, colors='black', linewidths=1)
plt.clabel(c2, levels_thetaz2, fmt='%1.2i')

figname='./figs/t_theta_zmean_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Zoom sur le vent zonal équatorial (QBO)

In [ ]:
print(fu)
fu_eq=fu.sel(lat=0).mean('lon')
u_eq=fu_eq['uwnd'].transpose()

levels_u = np.arange(-30, 35, 5)

fig = plt.figure(figsize=(15, 10))
fig.suptitle('', fontsize=16)
ax=fig.add_subplot(111)
ax.set_title('Zonal wind at the equator - zonal mean : NCEP '+year1+'-'+year2, fontsize=14)
ax.set_yscale('symlog')
ax.set_xlabel('Year')
ax.set_ylabel('Pressure level (hPa)')
ax.set_yticklabels(np.arange(70, 0, -10))
ax.set_ylim(70, 10)
ax.set_yticks(np.arange(70, 0, -10))
cf = ax.contourf(fu.time.values, lev, u_eq, levels_u, cmap='coolwarm', extend='both')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('m/s', size='small')

figname='./figs/u_zmean_eq_qbo'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
#Cartes en moyenne annuelle
fuyear=xr.open_dataset(diri+"uwnd.mon.mean.nc").sel(time=slice(year1,year2)).groupby('time.year').mean('time')
uyear = fuyear['uwnd']

#moyenne zonale
uyearz=uyear.mean(axis=3)

#variance temporelle (np.var)
uyear_var=np.var(uyearz,axis=0)

print(uyear_var.shape)

In [ ]:
levels_var = np.arange(0,160,5)

fig = plt.figure(figsize=(15, 10))
fig.suptitle('', fontsize=16)
ax=fig.add_subplot(111)
ax.set_title('Interannual variability of zonal wind - NCEP '+year1+'-'+year2, fontsize=14)
plot_zonal_mean2(ax)
cf = ax.contourf(lat, lev, uyear_var, levels_var, cmap='jet', extend='both')
#plt.clabel(c, levels, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Variance (m$^2$/s$^2$)', size='small')

figname='./figs/u_zmean_year_variance'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Vent méridien et vitesse verticale

In [ ]:
fig = plt.figure(figsize=(15., 10.))
ax = fig.add_subplot(1, 1, 1)
fig.suptitle('Vertical velocity (Pa/s), meridional wind (m/s) and zonal wind (m/s) - zonal mean : NCEP '+year1+'-'+year2, fontsize=12)

ax.set_title('Annual mean', fontsize=8)
plot_zonal_mean(ax)
ax.set_xlim(-60, 60)
cf = ax.contourf(lat, levw, wz_annual[:,:], levels_wz, cmap='seismic', extend='both')
c = ax.contour(lat, lev, vz_annual[:,:], levels_vz, colors='black', linewidths=1)
plt.clabel(c, levels_vz, fmt='%1.1f')
c2 = ax.contour(lat, lev, uz_annual[:,:], levels_uz_an, colors='green', linewidths=0.5)
plt.clabel(c2, levels_uz_an, fmt='%2.1i')

cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.75, pad=0.15)
cb.set_label('Pa/s', size='small')

figname='./figs/wv_zmean_annual_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=2, figsize=(15, 8), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Vertical velocity (Pa/s) and meridional wind (m/s) - zonal mean : NCEP '+year1+'-'+year2, fontsize=12)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=8)
    plot_zonal_mean(ax)
    ax.set_xlim(-60, 60)
    cf = ax.contourf(lat, levw, wz[i,:,:], levels_wz, cmap='seismic', extend='both')
    c = ax.contour(lat, lev, vz[i,:,:], levels_vz, colors='black', linewidths=1)
    plt.clabel(c, levels_vz, fmt='%1.1f')

cb = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=1, pad=0)
cb.ax.tick_params(labelsize=8) 
cb.set_label('Pa/s', size='small')

figname='./figs/wv_zmean_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Humidité et précipitations

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=1, figsize=(15, 15), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Specific humidity - zonal mean : '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    ax.set_yscale('symlog')
    ax.set_yticklabels(np.arange(1000, 300, -100))
    ax.set_ylim(1000, 300)
    ax.set_yticks(np.arange(1000, 300, -100))  
    ax.set_xticklabels(np.arange(-90, 100, 10))
    ax.set_xticks(np.arange(-90, 100, 10))    
    cf = ax.contourf(lat, levq, qz[i,:,:], levels_qz, cmap='jet', extend='both')
    c = ax.contour(lat, levq, qz[i,:,:], levels_qz, colors='black', linewidths=1)
    plt.clabel(c, levels_qz, fmt='%2.1i')
 
cf = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cf.set_label('g/kg', size='small')

fig.set_constrained_layout_pads(w_pad=0., h_pad=0.1, hspace=0., wspace=0.)


figname='./figs/q_zmean_climatology'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))
plt.title('Precipitation - zonal mean : GPCP '+year1+'-'+year2)
plt.xlabel('Latitude')
plt.ylabel('Precipitation (mm/day)')
#plt.xticks(np.arange(-90, 120, 30), ('90S', '60S', '30S', 'Eq', '30N', '60N', '90N'))
plt.xticks(np.arange(-90, 120, 30))

plt.plot(lats, prz_annual, label='Annual')

for i in range(4):
 plt.plot(lats, prz[i,:], label=seasons[i])
 plt.axhline(0, color='black', linestyle="--")
 plt.axvline(0, color='black', linestyle="--")

plt.legend()

figname='./figs/precip_zmean_climatology'
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()